In [2]:

import numpy as np
import pandas as pd

# sklearn
import sklearn as sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.metrics import roc_auc_score
from sklearn.cluster import KMeans

# Qiskit
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.circuit.library import ZZFeatureMap, PauliFeatureMap, ZFeatureMap
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityStatevectorKernel
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit.primitives import Sampler
from qiskit.circuit import ParameterVector, QuantumCircuit, Parameter
from qiskit import execute, Aer, IBMQ, QuantumRegister, ClassicalRegister, BasicAer
from qiskit.utils import QuantumInstance

import warnings
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

# Load the dataset
df = pd.read_csv('yourdataset.csv')

# Separate X and y, dropping non-useful columns
X = df.drop(['target'], axis="columns")
y = df['target']

# Split train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Detect constant coolumns to be deleted
constant_columns = X_train.columns[X_train.nunique() == 1]
constant_columns

# Drop the constant columns from all sets
X_train = X_train.drop(columns=constant_columns)
X_test = X_test.drop(columns=constant_columns)

# Calculate the correlation of each feature with the target variable
correlations = X_train.corrwith(y_train)

# Number of dimensions
num_dimensions = 2

# Reshape the correlations into a 2D array
correlations_reshaped = np.reshape(correlations.values, (-1, 1))

# Fit the k-means algorithm to the reshaped correlations
kmeans = KMeans(n_clusters=num_dimensions, random_state=0).fit(correlations_reshaped)

# Assign each feature to a cluster
clusters = kmeans.labels_

# Split the features into groups based on the cluster assignments
groups = [np.where(clusters == i)[0] for i in range(num_dimensions)]

# Apply LDA to each group of features to create a new feature
features_lda = np.empty((X_train.shape[0], num_dimensions))
features_lda_test = np.empty((X_test.shape[0], num_dimensions))
for i, group in enumerate(groups):
    lda = LDA(n_components=1)
    features_lda[:, i] = lda.fit_transform(X_train.iloc[:, group], y_train).ravel()
    features_lda_test[:, i] = lda.transform(X_test.iloc[:, group]).ravel()

# Normalization
minmax_scaler = MinMaxScaler().fit(features_lda)
X_train_qsvc = minmax_scaler.transform(features_lda)
X_test_qsvc = minmax_scaler.transform(features_lda_test)

# Secure dimensionality across different methods
features_dim = num_dimensions
num_params = features_dim
num_features = features_dim
num_inputs = features_dim
num_qubits = features_dim

#Math Part
import numpy as np
import pandas as pd

#Graph
import matplotlib.pyplot as plt

import importlib
import circuit
import encoding
import qsvm
import fitness
import gsvm
importlib.reload(circuit)
importlib.reload(encoding)
importlib.reload(qsvm)
importlib.reload(fitness)
importlib.reload(gsvm)

import pandas as pd
import time

def evol(output="output.csv"):
    y = y_train
    X = X_train_qsvc
    start = time.time()
    pop, pareto, logbook = gsvm.gsvm(nqubits=num_dimensions, depth=6, nparameters=num_dimensions,
                                     X=X, y=y, weights=[-1.0,1.0],
                                     mu=50,lambda_=20, ngen=100)
    print(f'Simulation finished after {time.time()-start} seconds')
    print(f'\nGenetic-algorithm output ({output})')
    #print('generation, individual, gates/qubit, accuracy')
    print('---------------------------------------------')
    with open(output, "w") as f:
        for ide, ind in enumerate(pareto):
            genes=''.join(str(i) for i in list(ind))
            gates, acc = ind.fitness.values
            line = f'{ide},"{genes}",{gates},{acc}'
            f.write(line)
            f.write('\n')
            print(line)
evol()

import encoding as e2
import random

iot_result = pd.read_csv('output.csv',header=None)

def ordenar_salidas_pareto(dataframe):
    dataframe.columns=['ind','circ','gates','acc']
    dataframe.sort_values(['acc','gates'], ascending=[False,False],inplace=True)
    dataframe.reset_index(inplace=True)
    dataframe.pop('index')
    return dataframe

iot_salidas = ordenar_salidas_pareto(iot_result)
iot_salidas

from qiskit.circuit import ParameterVector, QuantumCircuit, Parameter
from qiskit import execute, Aer, IBMQ, QuantumRegister, ClassicalRegister, BasicAer
from qiskit.utils import QuantumInstance
def coding_bits(b):
    c = [b[n:n+5] for n,i in enumerate(b) if n%5==0]
    c_p=[]
    coding_0=[]
    for i in range(len(c)):
        for j in c[i]:
            c_p.append(str(j))
    np.asarray(c_p)
    c = [c_p[n:n+5] for n,i in enumerate(c_p) if n%5==0]
    for i in c:
        coding_0.append(''.join(i))
    return coding_0

class CircuitConversor:

    def __init__(self, nqubits, nparameters):
        gates = {}
        for n, suffix in enumerate(['00','01','10','11']):
            angle = np.pi / (2**n)
            gates['000'+suffix] = (self.make_H(), 1.0)
            gates['001'+suffix] = (self.make_cx(), 2.0)
            gates['010'+suffix] = (self.make_id(), 0.0)
            gates['011'+suffix] = (self.make_rx(angle), 1.0)
            gates['100'+suffix] = (self.make_rz(angle), 1.0)
            gates['101'+suffix] = (self.make_id(), 0.0)
            gates['110'+suffix] = (self.make_id(), 0.0)
            gates['111'+suffix] = (self.make_ry(angle), 1.0)
        self.gates = gates
        self.nqubits = nqubits
        self.register = QuantumRegister(nqubits, 'q')
        self.nparameters = nparameters
        self.nqubits = nqubits
        self.x = ParameterVector('x', nparameters)

    def __call__(self, coding_0):
        print(coding_0)
        circuit = QuantumCircuit(self.register)
        k = 0
        cost = 0
        for ndx, z in enumerate(coding_0):
            qubit = ndx % self.nqubits
            target = (ndx + 1) % self.nqubits
            fn, weight = self.gates[z]
            k = fn(circuit, k, qubit, target)
            cost += weight
        for i in range(k, self.nparameters):
            circuit.rz(self.x[i]*0, self.register[0])
        return circuit, cost

    def make_id(self):
        def operation(circuit, k, qubit, target):
            return k
        return operation

    def make_H(self):
        def operation(circuit, k, qubit, target):
            circuit.h(self.register[qubit])
            return k
        return operation

    def make_cx(self):
        def operation(circuit, k, qubit, target):
            circuit.cx(self.register[qubit], self.register[target])
            return k
        return operation

    def make_rx(self, angle):
        def operation(circuit, k, qubit, target):
            circuit.rx(self.x[k%self.nparameters] * angle,
                       self.register[qubit])
            return k+1
        return operation

    def make_ry(self, angle):
        def operation(circuit, k, qubit, target):
            circuit.ry(self.x[k%self.nparameters] * angle,
                       self.register[qubit])
            return k+1
        return operation

    def make_rz(self, angle):
        def operation(circuit, k, qubit, target):
            circuit.rz(self.x[k%self.nparameters] * angle,
                       self.register[qubit])
            return k+1
        return operation

import os
import psutil

class Fitness:

    def __init__(self, nqubits, nparameters, X, y, quantum_instance):
        self.nqubits = nqubits
        self.nparameters = nparameters
        self.cc = CircuitConversor(nqubits, nparameters)
        self.instance = quantum_instance
        self.X = X
        self.y = y

    def __call__(self, POP):
        try:
            return self.fitness(POP)
        except Exception as e:
            print(f'Exception happened during fitness():\n  {e}')
            process = psutil.Process(os.getpid())
            print(f'  RUSAGE_SELF: {process.memory_info()}')
        return 1000, 100000.0

    def fitness(self, POP):
        print('Invoked fitness')
        #Convertimos el individuo en el fenotipo (ansatz)
        fm, puertas = self.cc(coding_bits(POP))
        
cc = CircuitConversor(nqubits=num_dimensions, nparameters=num_dimensions)
fm_1, puertas_1 = cc(coding_bits(iot_salidas.circ[0]))

print(puertas_1)

# Declare the sampler
sampler = Sampler()

# Declare fidelity
fidelity = ComputeUncompute(sampler=sampler)

# Define quantum kernel statevector
qkernel = FidelityStatevectorKernel(feature_map=fm_1)

# Model
qsvc = QSVC(quantum_kernel=qkernel, probability=True, C=0.1)

# Fit the model 
qsvc.fit(X_train_qsvc, y_train)

# Use predict_proba to get the predicted probabilities for the positive class
predictions_qsvc = qsvc.predict_proba(X_test_qsvc)[:, 1]

# Compute the AUC using the true labels and the predicted probabilities
auc_test = roc_auc_score(y_test, predictions_qsvc)

print("AUC for the test set: ", auc_test)

multi
String: 110110010101000100100111011101010101110100101011001010110111
 -> accuracy = 0.9882352941176471, gates = 9.0
String: 000110111010011111001001100001100001100000000001010011011101
 -> accuracy = 0.9882352941176471, gates = 13.0
String: 111001001101001011000010011110000111101010101001111101011100
 -> accuracy = 0.9899159663865547, gates = 10.0
String: 011011101101011011010101101001011011001101110001001111010001
 -> accuracy = 0.9885714285714285, gates = 9.0
String: 111010101111001001011110011001011110110010000101111111000110
 -> accuracy = 0.9872268907563025, gates = 10.0
String: 000101110010000011111010101100101110101000000111100110111010
 -> accuracy = 0.9872268907563025, gates = 8.0
String: 101110010011001001100000111100011010110001010010011011001011
 -> accuracy = 0.98890756302521, gates = 8.0
String: 011101100000010110001100100101000001101100110101000011101110
 -> accuracy = 0.9899159663865547, gates = 10.0
String: 01111011010010101000000111101111001111110011000001001101